# 组件解析：Agent

[文档地址](https://ai.feishu.cn/wiki/IqMMwRVv4itfZmkOnaBcaagPnof?from=from_copylink)


In [8]:
# 配置开发环境
import os
from dotenv import load_dotenv
from pathlib import Path
from loguru import logger


load_dotenv()
root_path = Path.cwd().parent.parent

str(root_path), os.getenv("OPENAI_BASE_URL"), os.getenv("OPENAI_API_KEY")

('/home/zht/projects/Agent',
 'https://api.openai-proxy.org/v1',
 'sk-A7irm5LFwUUuMHPQ58V5qnygQQEYOGm4J8bfiUo5jm8N0lai')

# 1. 模型


**静态模型**

静态模型是最常用的方法，下面构建一个静态模型，并以该模型为基础构建一个代理：


In [6]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.messages import HumanMessage

model = init_chat_model("openai:gpt-4o-mini")

agent = create_agent(
    model = model
)

resp = agent.invoke(input = {"messages": [HumanMessage(content="你好")]})
resp["messages"][-1].content

'你好！有什么我可以帮助你的吗？'

**动态模型**

有时候可能希望对不同情况使用不同的模型，我们叫做动态模型调用。LangChain 可以通过 @wrap_model_call 中间件实现模型动态调用。


In [ ]:
from langchain.chat_models import init_chat_model
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse


basic_model = init_chat_model("openai:gpt-4o-mini")
advanced_model = init_chat_model("openai:gpt-4o")

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """ 根据对话复杂性选择模型。 """
    msg_cnt = len(request.messages)  # 或者 request.state["messages"]

    if msg_cnt > 3:
        model = advanced_model
    else:
        model = basic_model
    
    request.model = model
    logger.info(f"Using model: {model.model_name}")
    return handler(request)


agent = create_agent(
    model = basic_model,
    middleware = [dynamic_model_selection]
)


In [10]:
# 构建只有一条的消息列表
messages = [HumanMessage(content="你好")]

resp = agent.invoke(input = {"messages": messages})
resp["messages"][-1].content


/tmp/ipykernel_2772/3327199740.py:18: DeprecationWarning: Direct attribute assignment to ModelRequest.model is deprecated. Use request.override(model=...) instead to create a new request with the modified attribute.
  request.model = model
2025-12-01 09:18:13.064 | INFO     | __main__:dynamic_model_selection:19 - Using model: gpt-4o-mini


'你好！有什么我可以帮助你的吗？'

In [11]:
# 构建有五条消息的消息列表
messages = [
    HumanMessage(content="你好"),
    HumanMessage(content="你叫什么名字？"),
    HumanMessage(content="你今年多大了？"),
    HumanMessage(content="你住在哪里？"),
    HumanMessage(content="你最喜欢什么？"),
]

resp = agent.invoke(input = {"messages": messages})
resp["messages"][-1].content


/tmp/ipykernel_2772/3327199740.py:18: DeprecationWarning: Direct attribute assignment to ModelRequest.model is deprecated. Use request.override(model=...) instead to create a new request with the modified attribute.
  request.model = model
2025-12-01 09:18:57.408 | INFO     | __main__:dynamic_model_selection:19 - Using model: gpt-4o


'你好！我是一个人工智能助手，没有名字、年龄或住所。我的主要职责是帮助你回答问题和提供信息。如果你有任何问题或需要帮助，请告诉我！'

# 2. 工具

通过 `@tool` 装饰器可以创建工具。


In [12]:
from langchain.tools import tool
from langchain.agents import create_agent


@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"


@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    return f"Weather in {location}: Sunny, 72°F"


agent = create_agent(init_chat_model("openai:gpt-4o-mini"), tools=[search, get_weather])

resp = agent.invoke(
    {"messages": [HumanMessage(content="太原天气怎么样？")]}
)
resp["messages"][-1].content


'太原的天气是晴天，气温为72°F（约22°C）。'

工具调用可能会出错，从而触发 exception，可以通过中间件 `@wrap_tool_call` 对工具调用进行控制，并处理其返回的 exception。


In [16]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage
from langchain.tools import tool  


@tool
def search(query: str) -> str:
    """ 搜索信息 """
    # pretend to raise an error 
    raise Exception("I am a tool error")
    return f"Results for: {query}"


@tool
def get_weather(location: str) -> str:
    """ 获取天气信息 """
    return f"Weather in {location}: Sunny, 72°F"


@wrap_tool_call
def handle_tool_errors(request, handler):
    """ 处理工具执行错误 """
    try:
        return handler(request)
    except Exception as e:
        # 返回一个工具错误消息
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )


agent = create_agent(
    model = init_chat_model("openai:gpt-4o-mini"),
    tools = [search, get_weather],
    middleware = [handle_tool_errors],  # 用于处理工具执行错误的中间件
)

resp = agent.invoke(
    {"messages": [HumanMessage(content="搜索一下太原的具体位置。")]}
)

for msg in resp["messages"]:
    print(msg.content)


搜索一下太原的具体位置。

Tool error: Please check your input and try again. (I am a tool error)

Tool error: Please check your input and try again. (I am a tool error)

Tool error: Please check your input and try again. (I am a tool error)
我无法获取关于太原的具体位置信息，但我可以告诉你太原是中国山西省的省会，位于中国北部的黄土高原上。如果你需要更详细的位置或地理信息，可以考虑使用地图服务。


# 系统提示



In [18]:
from dataclasses import dataclass
from langchain.agents.middleware import dynamic_prompt, ModelRequest


@dataclass
class Context():
    user_role: str


@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    user_role = request.runtime.context.user_role
    base_prompt = "You are a helpful assistant."

    if user_role == "expert":
        return f"{base_prompt} Provide detailed technical responses."
    elif user_role == "beginner":
        return f"{base_prompt} Explain concepts simply and avoid jargon."

    return base_prompt


model = init_chat_model("openai:gpt-4o-mini")
agent = create_agent(
    model = model,
    middleware = [user_role_prompt],
    context_schema = Context
)

resp = agent.invoke(
    input = {
        "messages": [HumanMessage(content="解释一下什么是 langchain?")]
    },
    context = {"user_role": "beginner"}
)

resp["messages"][-1].content


'LangChain 是一个用于构建应用程序的框架，尤其是那些利用自然语言处理和大语言模型（像 GPT-3 或 ChatGPT）进行交互的应用程序。简单来说，它让开发者更容易创建能够理解和生成自然语言的程序。\n\nLangChain 提供了一些工具和功能，帮助开发者处理文本、创建对话系统、生成响应等。它可以将多种数据源（例如数据库、API 等）和大语言模型结合起来，构建复杂的应用。\n\n总的来说，LangChain 是一个让开发者在使用大语言模型时更加高效和方便的工具，使他们能够快速构建出各种有趣和实用的应用。'

# 结构化输出


**工具策略结构化输出**


In [ ]:
from pydantic import BaseModel
from langchain.agents.structured_output import ToolStrategy


class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str


agent = create_agent(
    model = "openai:gpt-4o-mini",
    response_format = ToolStrategy(ContactInfo)
)

msg_content = """
帮我从下面的字符串中提取联系人信息：
```
琼恩雪诺，电话号为135746427798，邮箱为jon@snow.com
```
"""

resp = agent.invoke(
    input = {"messages": [HumanMessage(content = msg_content)]}
)


print(resp["messages"][-1].content)  # 查看最后一条消息
print(resp["structured_response"])  # 查看结构化输出

# 查看所有内容
for msg in resp["messages"]:
    print(msg)


Returning structured response: name='琼恩雪诺' email='jon@snow.com' phone='135746427798'
name='琼恩雪诺' email='jon@snow.com' phone='135746427798'
content='\n帮我从下面的字符串中提取联系人信息：\n```\n琼恩雪诺，电话号为135746427798，邮箱为jon@snow.com\n```\n' additional_kwargs={} response_metadata={} id='e873b1ef-be46-4906-8cb2-9bdda4686652'
content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 83, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-ChnYMi2uA4HpTb2eBWKBbxi5lHXCC', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--1a3b2757-3a28-44c8-a378-980a7fb7b1f0-0' tool_calls=[{'name': 'ContactInfo', 'args': {'name': '琼恩雪诺', 'email': 'jon@snow.com'